# Quantitative Value Trading

### Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import warnings
from scipy import stats
from statistics import mean

### Importing List of Stocks
Using s&p500_updated as it is the updated list of s&p500 companies. 

In [2]:
stocks = pd.read_csv("s&p500_updated.csv")

### Acquiring an API Token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

### Adding Stocks Data to Pandas Data Frame

In [4]:
my_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio', 'PE Score',
              'Price-to-Book Ratio', 'PB Score', 'Price-to-Sales Ratio', 'PS Score', 'EV/EBITDA',
              'EV/EBITDA Score', 'EV/GP', 'EV/GP Score', 'RV Score']
dataframe = pd.DataFrame(columns=my_columns)

### To avoid the FutureWarning when appending onto the pd DataFrame

In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

### Making an API Call (testing 1)

1) base url get it from IEX Cloud documentation <br>
2) we want the **"quote" and "advanced-stats"** endpoint from the IEX Cloud API because this gives us: <br>
    - Price of each stock  <br>
    - Price-to-earnings ratio <br>
    - Price-to-book ratio <br>
    - Price-to-sales ratio <br>
    - Enterprise Value (total value of company to buy it off including all the stocks and debts, etc.) divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA) <br>
    - Enterprise Value divided by Gross Profit (EV/GP)

In [6]:
symbol = 'GOOG'
api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
print(data.status_code)

200


### Using Batch API Calls to improve performance
Splitting the 500 into 5 batches to make 5 API calls instead of 500. <br>

In [7]:
'''Function sourced from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks'''
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
ticker_groups = list(chunks(stocks['Ticker'], 100))
batch_apis = []
for i in range(len(ticker_groups)):
    batch_apis.append(",".join(ticker_groups[i]))

for batch_api in batch_apis:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={batch_api}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    tickers = batch_api.split(',')
    for ticker in tickers:
        price = data[ticker]['quote']['latestPrice']
        pe_Ratio = data[ticker]['quote']['peRatio']
        pb_Ratio = data[ticker]['advanced-stats']['priceToBook']
        ps_Ratio = data[ticker]['advanced-stats']['priceToSales']
        enterprise_value = data[ticker]['advanced-stats']['enterpriseValue']
        ebitda = data[ticker]['advanced-stats']['EBITDA']
        gross_profit = data[ticker]['advanced-stats']['grossProfit']
        try:
            evToebitda = enterprise_value / ebitda
        except TypeError:
            evToebitda = np.NaN
        try:
            evTogross_profit = enterprise_value / gross_profit
        except TypeError:
            evTogross_profit = np.NaN
            
        ticker_data = pd.Series([ticker, price, 'N/A', pe_Ratio, 'N/A', pb_Ratio, 'N/A', ps_Ratio, 'N/A', evToebitda, 
                                 'N/A', evTogross_profit,'N/A', 'N/A'], index=my_columns)
        dataframe = dataframe.append(ticker_data, ignore_index=True)

In [9]:
dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score
0,MMM,125.51,N/A,10.97,N/A,4.92,N/A,1.99,N/A,12.033548,N/A,5.092627,N/A,N/A
1,AOS,56.68,N/A,18.24,N/A,4.14,N/A,1.95,N/A,9.778294,N/A,5.192038,N/A,N/A
2,ABT,110.14,N/A,24.78,N/A,5.39,N/A,4.28,N/A,16.883275,N/A,7.530208,N/A,N/A
3,ABBV,165.58,N/A,21.46,N/A,18.51,N/A,5.1,N/A,12.063887,N/A,8.361723,N/A,N/A
4,ABMD,393.10,N/A,68.64,N/A,11.42,N/A,16.15,N/A,59.285925,N/A,18.462693,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,133.78,N/A,30.66,N/A,-4.3,N/A,5.64,N/A,21.611278,N/A,9.605622,N/A,N/A
498,ZBRA,250.42,N/A,28.21,N/A,4.79,N/A,2.29,N/A,14.332670,N/A,5.644480,N/A,N/A
499,ZBH,127.70,N/A,95.92,N/A,2.16,N/A,3.68,N/A,19.538797,N/A,6.228950,N/A,N/A
500,ZION,46.16,N/A,5.22,N/A,1.65,N/A,1.88,N/A,4.191663,N/A,1.944259,N/A,N/A


### Filling in the missing values from the table (NaN values)

1) Creating a new dataframe which identifies the missing values. (Using pandas, `isnull` method) <br>
2) Filling in the remaining (using pandas `fillna` method) <br>
3) Filling in the missing values with the mean from the column <br>

Implementing a screening process. If there are less than 20 columns with 'None' type values then instead of approximating with mean we will remove them to ensure the data remains as accurate as possible. This is because at the end we will only consider the top 50 stocks for an equal weight portfolio.

In [10]:
dataframe1 = dataframe[dataframe.isnull().any(axis=1)] #This searches for all the none types in the dataframe 
if len(dataframe1) <= 20:
    dataframe = dataframe.drop(dataframe1.index) #removes all the none type values from our database
else:
    columns = ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']
    for column in columns:
        mean_val = dataframe[column].mean()
        dataframe[column].fillna(mean_val, inplace=True)

In [11]:
dataframe[dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score


### Calculating the Scores

Using the `percentileofscore` method to give each ticker's weighting depending on it's metric. I.e. if given 20% then 20% of the companies score below this one. <br>
We do 100 minus the percentile to get score i.e. 20% (percentile) corresponds to a score of 80 as its in the top 80%). <br>
The reason for using score becomes clear later on (to make the portfolio RV weighted).

In [12]:
metrics = {'Price-to-Earnings Ratio': 'PE Score', 'Price-to-Book Ratio':'PB Score',
           'Price-to-Sales Ratio': 'PS Score', 'EV/EBITDA':'EV/EBITDA Score', 'EV/GP':'EV/GP Score'}

for index in dataframe.index:
    for metric in metrics.keys():
        dataframe.loc[index, metrics[metric]] = 100 - stats.percentileofscore(dataframe[metric], dataframe.loc[index, metric])

dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score
0,MMM,125.51,N/A,10.97,78.483607,4.92,32.479508,1.99,62.909836,12.033548,58.401639,5.092627,62.909836,N/A
1,AOS,56.68,N/A,18.24,58.606557,4.14,39.344262,1.95,63.52459,9.778294,68.647541,5.192038,61.47541,N/A
2,ABT,110.14,N/A,24.78,39.754098,5.39,29.405738,4.28,27.151639,16.883275,30.942623,7.530208,40.163934,N/A
3,ABBV,165.58,N/A,21.46,47.95082,18.51,5.327869,5.1,21.311475,12.063887,57.786885,8.361723,34.221311,N/A
4,ABMD,393.10,N/A,68.64,6.147541,11.42,10.040984,16.15,0.819672,59.285925,1.434426,18.462693,3.278689,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,133.78,N/A,30.66,26.02459,-4.3,93.237705,5.64,18.647541,21.611278,15.368852,9.605622,25.819672,N/A
498,ZBRA,250.42,N/A,28.21,30.737705,4.79,33.913934,2.29,55.635246,14.332670,44.262295,5.644480,56.762295,N/A
499,ZBH,127.70,N/A,95.92,3.278689,2.16,63.729508,3.68,33.401639,19.538797,21.311475,6.228950,50.614754,N/A
500,ZION,46.16,N/A,5.22,91.803279,1.65,76.741803,1.88,65.368852,4.191663,93.237705,1.944259,88.52459,N/A


The RV score stands for Robust Value. It is the mean of the 4 percentiles we have calculated.

In [13]:
for index in dataframe.index:
    percentiles = [dataframe.at[index, x] for x in list(metrics.values())]
    dataframe.loc[index, 'RV Score'] = mean(percentiles)
dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score
0,MMM,125.51,N/A,10.97,78.483607,4.92,32.479508,1.99,62.909836,12.033548,58.401639,5.092627,62.909836,59.036885
1,AOS,56.68,N/A,18.24,58.606557,4.14,39.344262,1.95,63.52459,9.778294,68.647541,5.192038,61.47541,58.319672
2,ABT,110.14,N/A,24.78,39.754098,5.39,29.405738,4.28,27.151639,16.883275,30.942623,7.530208,40.163934,33.483607
3,ABBV,165.58,N/A,21.46,47.95082,18.51,5.327869,5.1,21.311475,12.063887,57.786885,8.361723,34.221311,33.319672
4,ABMD,393.10,N/A,68.64,6.147541,11.42,10.040984,16.15,0.819672,59.285925,1.434426,18.462693,3.278689,4.344262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,133.78,N/A,30.66,26.02459,-4.3,93.237705,5.64,18.647541,21.611278,15.368852,9.605622,25.819672,35.819672
498,ZBRA,250.42,N/A,28.21,30.737705,4.79,33.913934,2.29,55.635246,14.332670,44.262295,5.644480,56.762295,44.262295
499,ZBH,127.70,N/A,95.92,3.278689,2.16,63.729508,3.68,33.401639,19.538797,21.311475,6.228950,50.614754,34.467213
500,ZION,46.16,N/A,5.22,91.803279,1.65,76.741803,1.88,65.368852,4.191663,93.237705,1.944259,88.52459,83.135246


### Selecting top 50 and calculating Number of Shares

We will choose the top 50 with the highest RV Scores. For example take price to earnings in consideration. If a company has a low PE Ratio means that the price of the stock is lower compared to the earnings made by the company for that stock. I.e. it's has a price that it's intrinsic value. However as all the percentiles were flipped to give us the scores, we now consider the top 50 <br>
Instead of giving each stock an equal weighting, to determine how many shares to buy, I have used the RV Score to give me a weighting. The higher the RV Score the more we will buy.

In [14]:
dataframe.sort_values(by='RV Score', ascending=False, inplace=True) #to get top 50 with highest RV Score
dataframe = dataframe[:50]
dataframe.reset_index(drop=True, inplace=True)
dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score
0,GM,36.40,N/A,6.07,89.754098,0.774,90.983607,0.3449,95.696721,1.825045,98.360656,1.465072,91.188525,93.196721
1,AIG,63.10,N/A,3.84,94.467213,1.25,85.040984,0.7818,88.114754,2.104161,98.155738,0.751137,97.540984,92.663934
2,F,12.04,N/A,5.17,92.213115,1.1,87.090164,0.3073,96.516393,3.467796,95.696721,1.518140,90.778689,92.459016
3,COF,91.60,N/A,4.42,93.545082,0.6654,91.393443,0.953,83.606557,2.980064,97.540984,1.013662,95.286885,92.27459
4,SYF,34.09,N/A,5.34,91.598361,1.23,85.245902,0.8244,86.680328,3.006012,97.131148,0.932033,96.516393,91.434426
5,C,44.83,N/A,6.1,89.54918,0.4741,91.803279,0.9888,82.991803,3.705422,94.877049,0.999169,95.696721,90.983607
6,LUMN,5.45,N/A,2.77,94.672131,0.4454,92.008197,0.3012,96.92623,4.089916,93.852459,2.900465,77.459016,90.983607
7,BA,190.17,N/A,-13.51,96.721311,-6.35,93.647541,1.9,64.54918,-29.755340,99.385246,-188.519019,99.795082,90.819672
8,LNC,29.42,N/A,-2.56,95.491803,2.31,59.528689,0.2643,97.95082,-2.841822,98.565574,0.255147,99.590164,90.22541
9,NRG,31.85,N/A,4.21,94.057377,1.48,80.327869,0.2411,98.565574,3.831714,94.262295,2.836254,79.303279,89.303279


In [15]:
val = input("How much money would you like to invest? ($) ")
value = []
def money(val):
    try:
        num = float(val)
    except:
        num = "error"
        
    if type(num) == float:
        print("Thank you")
        value.append(num)
    else:
        val = input("Please input a number ")
        return money(val)

money(val)

How much money would you like to invest? ($) 100000
Thank you


In [16]:
invest = value[0]

sum_RV = sum(dataframe['RV Score'])

for index in dataframe.index:
    dataframe.loc[index,'Weighting'] = dataframe.at[index, 'RV Score'] / sum_RV
        
print(sum(dataframe['Weighting'])) # If the sum is 1 then it is correctly weighted and I can base my portfolio off this

def RV_weighted_portfolio(invest):
    ''''Calculates how many shares to get of each stock'''
    for i in range(0, 50):
        ratio = invest * dataframe.at[i, 'Weighting']
        dataframe.loc[i, 'Number of Shares to Buy'] = ratio // dataframe['Price'][i]
    return dataframe

RV_weighted_portfolio(invest)

1.0


/var/folders/lh/zhzx5c3n5csbpf2_0sskj1zr0000gn/T/ipykernel_42716/2270311094.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.loc[index,'Weighting'] = dataframe.at[index, 'RV Score'] / sum_RV


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Score,Price-to-Book Ratio,PB Score,Price-to-Sales Ratio,PS Score,EV/EBITDA,EV/EBITDA Score,EV/GP,EV/GP Score,RV Score,Weighting
0,GM,36.40,59.0,6.07,89.754098,0.774,90.983607,0.3449,95.696721,1.825045,98.360656,1.465072,91.188525,93.196721,0.021658
1,AIG,63.10,34.0,3.84,94.467213,1.25,85.040984,0.7818,88.114754,2.104161,98.155738,0.751137,97.540984,92.663934,0.021534
2,F,12.04,178.0,5.17,92.213115,1.1,87.090164,0.3073,96.516393,3.467796,95.696721,1.518140,90.778689,92.459016,0.021487
3,COF,91.60,23.0,4.42,93.545082,0.6654,91.393443,0.953,83.606557,2.980064,97.540984,1.013662,95.286885,92.27459,0.021444
4,SYF,34.09,62.0,5.34,91.598361,1.23,85.245902,0.8244,86.680328,3.006012,97.131148,0.932033,96.516393,91.434426,0.021249
5,C,44.83,47.0,6.1,89.54918,0.4741,91.803279,0.9888,82.991803,3.705422,94.877049,0.999169,95.696721,90.983607,0.021144
6,LUMN,5.45,387.0,2.77,94.672131,0.4454,92.008197,0.3012,96.92623,4.089916,93.852459,2.900465,77.459016,90.983607,0.021144
7,BA,190.17,11.0,-13.51,96.721311,-6.35,93.647541,1.9,64.54918,-29.755340,99.385246,-188.519019,99.795082,90.819672,0.021106
8,LNC,29.42,71.0,-2.56,95.491803,2.31,59.528689,0.2643,97.95082,-2.841822,98.565574,0.255147,99.590164,90.22541,0.020968
9,NRG,31.85,65.0,4.21,94.057377,1.48,80.327869,0.2411,98.565574,3.831714,94.262295,2.836254,79.303279,89.303279,0.020753


### Using the xlsxwriter to format the output into an excel file

In [17]:
writer = pd.ExcelWriter("Portfolio.xlsx", engine='xlsxwriter')
dataframe.to_excel(writer, sheet_name="Portfolio", index=False)

In [18]:
background_color = '#FFFFFF'
font_color = '#000000'

string_format = writer.book.add_format({'font_color': font_color, 'bg_color': background_color})

dollar_format = writer.book.add_format({'num_format':'$0.00', 'font_color': font_color, 
                                        'bg_color': background_color})

integer_format = writer.book.add_format({'num_format':'0','font_color': font_color,
                                         'bg_color': background_color, 'border':1})

float_format = writer.book.add_format({'num_format':'0.0','font_color': font_color,
                                         'bg_color': background_color})

percent_format = writer.book.add_format({'num_format':'0%', 'font_color': font_color, 
                                           'bg_color': background_color})

In [19]:
column_formats = {'A': ['Ticker', string_format],
                  'B': ['Price', dollar_format],
                  'C': ['Number of Shares to Buy', integer_format],
                  'D': ['Price-to-Earnings Ratio', float_format],
                  'E': ['PE Score', percent_format],
                  'F': ['Price-to-Book Ratio', float_format],
                  'G': ['PB Score',percent_format],
                  'H': ['Price-to-Sales Ratio', float_format],
                  'I': ['PS Score', percent_format],
                  'J': ['EV/EBITDA', float_format],
                  'K': ['EV/EBITDA Score', percent_format],
                  'L': ['EV/GP', float_format],
                  'M': ['EV/GP Score', percent_format],
                  'N': ['RV Score', percent_format]}

for column in column_formats.keys():
    writer.sheets['Portfolio'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Portfolio'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [20]:
writer.save()